In [1]:
from google.colab import drive
drive.mount('/content/drive')

!git clone https://github.com/neccam/slt.git

%cd slt

!chmod +x data/download.sh
!data/download.sh

!cp ../drive/MyDrive/slt/requirements.txt requirements.txt

!pip -q install -r requirements.txt

Mounted at /content/drive
Cloning into 'slt'...
remote: Enumerating objects: 99, done.
remote: Total 99 (delta 0), reused 0 (delta 0), pack-reused 99
Unpacking objects: 100% (99/99), done.
/content/slt
--2021-01-04 13:47:05--  http://cihancamgoz.com/files/cvpr2020/phoenix14t.pami0.train
Resolving cihancamgoz.com (cihancamgoz.com)... 192.232.218.145
Connecting to cihancamgoz.com (cihancamgoz.com)|192.232.218.145|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1935289362 (1.8G)
Saving to: ‘phoenix14t.pami0.train’

phoenix14t.pami0.tr 100%[===================>]   1.80G  19.7MB/s    in 1m 40s  

2021-01-04 13:48:46 (18.4 MB/s) - ‘phoenix14t.pami0.train’ saved [1935289362/1935289362]

--2021-01-04 13:48:46--  http://cihancamgoz.com/files/cvpr2020/phoenix14t.pami0.dev
Resolving cihancamgoz.com (cihancamgoz.com)... 192.232.218.145
Connecting to cihancamgoz.com (cihancamgoz.com)|192.232.218.145|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13

In [2]:
!cp ../drive/MyDrive/slt/late-fusion/*.py signjoey/

In [3]:
!mkdir data/PHOENIX2014T 

!mv phoenix14t.pami0.dev data/PHOENIX2014T 
!mv phoenix14t.pami0.test data/PHOENIX2014T 
!mv phoenix14t.pami0.train data/PHOENIX2014T

!unzip -q '../drive/MyDrive/slt/keypoints/train_body_pose3d.zip' -d data
!unzip -q '../drive/MyDrive/slt/keypoints/train_face_pose3d.zip' -d data
!unzip -q '../drive/MyDrive/slt/keypoints/train_hand_pose3d.zip' -d data

!unzip -q '../drive/MyDrive/slt/keypoints/test_keypoints.zip' -d data
!unzip -q '../drive/MyDrive/slt/keypoints/dev_keypoints.zip' -d data

!unzip -q ../drive/MyDrive/slt/keypoints/dev_features.zip -d data
!unzip -q ../drive/MyDrive/slt/keypoints/train_features.zip -d data
!unzip -q ../drive/MyDrive/slt/keypoints/test_features.zip -d data

In [4]:
from six.moves import cPickle
import gzip
import torch
import os
import numpy as np
from tqdm import tqdm
import pickle

wd='data'

def load_dataset_file(filename):
    with gzip.open(filename, "rb") as f:
        loaded_object = pickle.load(f)
        return loaded_object

## Keypoints

In [5]:
body_parts = ['body_pose3d',
              'face_pose3d',
              'hand_pose3d']

for s in ['dev', 'test', 'train']:
  filename = "data/PHOENIX2014T/phoenix14t.pami0.{}".format(s)
  src_features = load_dataset_file(filename)
  dst_filename = "data/PHOENIX2014T/phoenix14t.pami1.{}".format(s)
  for src in tqdm(src_features):
    name = os.path.basename(src['name']) + '.npy'
    
    new_features_list = []

    for body_part in body_parts:
      dst = os.path.join(os.path.join(wd, s), os.path.join(body_part, name))

      new_features = np.load(dst)
      new_features = new_features.reshape((new_features.shape[0], np.prod(new_features.shape[1: ])))
      new_features_list.append(new_features)
      
    
    new_features = np.concatenate(new_features_list, axis=1)
    src['sign'] = torch.Tensor(new_features)

  with gzip.open(dst_filename, "wb") as f:
    pickle.dump(src_features, f)

100%|██████████| 7096/7096 [00:51<00:00, 136.98it/s]


## Hidden features

In [6]:
for s in ['dev', 'test', 'train']:
  filename = "data/PHOENIX2014T/phoenix14t.pami0.{}".format(s)
  dst_filename = "data/PHOENIX2014T/phoenix14t.pami2.{}".format(s)

  src_features = load_dataset_file(filename)

  for src in tqdm(src_features):
    name = os.path.basename(src['name']) + '.npy'

    dst = os.path.join(wd, os.path.join(s, name))
    
    new_features = np.load(dst)
    src['sign'] = torch.Tensor(new_features)

  with gzip.open(dst_filename, "wb") as f:
    pickle.dump(src_features, f)

100%|██████████| 7096/7096 [01:33<00:00, 75.56it/s]


## Change config files

In [7]:
import yaml
with open("configs/sign.yaml") as f:
     list_doc = yaml.load(f)

for s in ['train', 'test', 'dev']:
  list_doc['data'][s] = ['PHOENIX2014T/phoenix14t.pami{}.{}'.format(i, s) for i in range(3)]

list_doc['data']['feature_size'] = [1024, 417, 1024]
list_doc['model']['model_dir'] = "late_fusion"

with open("configs/sign.yaml", "w") as f:
    yaml.dump(list_doc, f)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  This is separate from the ipykernel package so we can avoid doing imports until


## Run

In [8]:
!cp -r '../drive/MyDrive/slt/models/baseline_1' baseline
!cp -r '../drive/MyDrive/slt/models/keypoints3d_1' keypoints3d
!cp -r '../drive/MyDrive/slt/models/hidden_features_1' features

In [9]:
import os
import sys
from signjoey.prediction import test

test(cfg_file="configs/sign.yaml", ckpts=["baseline/best.ckpt", "keypoints3d/best.ckpt", "features/best.ckpt"], output_path="")

2021-01-04 14:11:46,678 - ------------------------------------------------------------
2021-01-04 14:11:46,690 - [DEV] partition [RECOGNITION] experiment [BW]: 1
/usr/local/lib/python3.6/dist-packages/torchtext/data/field.py:359: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  var = torch.tensor(arr, dtype=self.dtype, device=device)
2021-01-04 14:12:20,953 - finished in 34.2631s 
2021-01-04 14:12:20,954 - ************************************************************
2021-01-04 14:12:20,955 - [DEV] partition [RECOGNITION] results:
	New Best CTC Decode Beam Size: 1
	WER 73.87	(DEL: 72.97,	INS: 0.13,	SUB: 0.77)
2021-01-04 14:12:20,956 - ************************************************************
2021-01-04 14:12:20,956 - ------------------------------------------------------------
2021-01-04 14:12:20,957 - [DEV] partition [RECOGNITION] exper

translation


2021-01-04 14:31:36,352 - [DEV] partition [Translation] results:
	New Best Translation Beam Size: 1 and Alpha: -1
	BLEU-4 20.10	(BLEU-1: 46.41,	BLEU-2: 32.57,	BLEU-3: 24.85,	BLEU-4: 20.10)
	CHRF 43.42	ROUGE 46.23
2021-01-04 14:31:36,353 - ------------------------------------------------------------
2021-01-04 14:45:24,078 - [DEV] partition [Translation] results:
	New Best Translation Beam Size: 4 and Alpha: 2
	BLEU-4 20.11	(BLEU-1: 44.69,	BLEU-2: 31.98,	BLEU-3: 24.67,	BLEU-4: 20.11)
	CHRF 42.14	ROUGE 46.23
2021-01-04 14:45:24,079 - ------------------------------------------------------------
2021-01-04 14:49:48,336 - [DEV] partition [Translation] results:
	New Best Translation Beam Size: 5 and Alpha: 2
	BLEU-4 20.22	(BLEU-1: 44.62,	BLEU-2: 31.94,	BLEU-3: 24.71,	BLEU-4: 20.22)
	CHRF 42.10	ROUGE 46.28
2021-01-04 14:49:48,337 - ------------------------------------------------------------
2021-01-04 14:59:13,772 - [DEV] partition [Translation] results:
	New Best Translation Beam Size: 7 an

In [ ]:
61.01 20.30 59.29 19.73